In [ ]:
!pip install whisper gradio transformers langchain faiss-cpu sentence-transformers python-dotenv requests
!pip install faiss-cpu

In [ ]:
!pip install langchain-community

In [ ]:
!pip install whisper

In [ ]:
!pip install python-dotenv
import os
from dotenv import load_dotenv

env_path = "/content/drive/MyDrive/alpha vantage.env"
load_dotenv(env_path)

api_key = os.getenv("S2K6PN06KDX573Y8")
print("✅ API Key loaded:", api_key)

notes_path = "/content/drive/MyDrive/finance_data.txt"
with open(notes_path, 'r') as f:


    notes_text = f.read()
print("📄 Notes Loaded:")
print(notes_text[:200])

In [5]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = notes_text.split('\n')
embeddings = model.encode(sentences)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

def search(query, top_k=2):
    q_embed = model.encode([query])
    D, I = index.search(np.array(q_embed), top_k)
    return [sentences[i] for i in I[0]]

ModuleNotFoundError: No module named 'faiss'

In [4]:
import requests

def get_stock_price(symbol):
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={S2K6PN06KDX573Y8}"
    response = requests.get(url).json()
    try:
        price = response["Global Quote"]["05. price"]
        return f"📈 Current price of {symbol}: ${price}"
    except:
        return "❌ Failed to fetch stock price."


In [3]:
def agent(query):
    context = search(query)
    if "price" in query or "stock" in query:
        symbol = query.split()[-1].upper()
        api_data = get_stock_price(symbol)
    else:
        api_data = ""

    response = f"🧠 Context from notes:\n" + "\n".join(context) + "\n\n" + api_data
    return response


In [2]:
import re

# Symbol map for major companies
symbol_map = {
    "tesla": "TSLA",
    "apple": "AAPL",
    "microsoft": "MSFT",
    "google": "GOOGL",
    "amazon": "AMZN",
    "meta": "META",
    "facebook": "META",
    "nvidia": "NVDA"
}

# Smart matcher using regex word boundaries
def extract_symbol(query):
    query = query.lower()
    for name in symbol_map:
        pattern = r"\b" + re.escape(name) + r"\b"
        if re.search(pattern, query):
            return symbol_map[name]
    return None


In [6]:
import requests
import os

def get_stock_price(symbol):
    api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}"
    r = requests.get(url)
    data = r.json()

    if "Global Quote" not in data or not data["Global Quote"]:
        return f"❌ Could not fetch price for {symbol}"

    quote = data["Global Quote"]
    price = quote.get("05. price", "N/A")
    change = quote.get("10. change percent", "N/A")

    return f"📈 {symbol} current price: ${price} ({change})"


In [7]:
def agent(query):
    try:
        # Step 1: Search RAG context
        context = search(query)

        # Step 2: Extract symbol
        symbol = extract_symbol(query)

        # Step 3: Get API response
        if symbol:
            api_response = get_stock_price(symbol)
        else:
            api_response = "❌ Couldn't detect company in your query."

        # Step 4: Format result
        result = f"🧠 Context from notes:\n" + "\n".join(context) + "\n\n" + api_response
        return result

    except Exception as e:
        return f"🚨 Error: {str(e)}"


In [ ]:

!pip install gradio
import gradio as gr

iface = gr.Interface(fn=agent, inputs="text", outputs="text", title="📊 AI Finance Agent")
iface.launch()
iface = gr.Interface(fn=agent, inputs="text", outputs="text", title="📊 AI Finance Agent", allow_flagging="never")
iface.launch(debug=True)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8dfe5891bd457c3d97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory 

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://708ca891e1481541b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
